In [1]:
import psycopg2  #import of the psycopg2 python library
import pandas as pd #import of the pandas python library
import pandas.io.sql as psql

##No transaction is started when commands are executed and no commit() or rollback() is required. 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# To connect to the server

In [2]:
try:
    # Connect to the postgreSQL server with username, and password credentials
    con = psycopg2.connect(user = "postgres",
                                  password = "ist",
                                  host = "127.0.0.1",
                                  port = "5432")
    
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    print("Connected Successfully to PostgreSQL server!!")
    
    # Obtain a DB Cursor to perform database operations
    cursor = con.cursor();
except (Exception, psycopg2.Error) as error :
     print ("Error while connecting to PostgreSQL", error)


Connected Successfully to PostgreSQL server!!


# To create a database

In [3]:
#DB_name variable    
name_Database   = "nobelprize_db";

# Create DB statement
sqlCreateDatabase = "CREATE DATABASE "+name_Database+";"

try:
    # Execute a SQL command: this creates a new DB
    cursor.execute(sqlCreateDatabase);
    print("Database '"+name_Database+"' Created Successfully!")
except (Exception, psycopg2.Error) as error :
    print("Error While Creating the DB: ",error)
    
finally:
    # Close communication with the database
    cursor.close() #to close the cusrsor
    con.close() #to close the connection/ we will open a new connection to the created DB

Error While Creating the DB:  FEL:  databas "nobelprize_db" finns redan



# To establish connection

In [4]:
# get a new connection but this time point to the created "tartupurchases" DB.
con = psycopg2.connect(user = "postgres",
                       password = "ist",
                       host = "127.0.0.1",
                       port = "5432",
                       database = "nobelprize_db")

try:
    # Obtain a new DB Cursor (to "tartupurchases" DB )
    cursor = con.cursor();
    print("connected again to the server, and cusor now on nobelprize_db!!")
except (Exception, psycopg2.Error) as error:
    print("Error in Connection",error)

connected again to the server, and cusor now on nobelprize_db!!


# Read the data from jsonfiles

In [5]:
import json
with open ('laureates.json','r') as f:
    laureatesList = json.loads(f.read())
with open ('nobel_prize.json','r') as f:
    prizeList = json.loads(f.read())

# Handling the data and divide into tables

In [22]:
winners = [] #winnerId, countryCode, motivationId
prizes = []  #year, category, motivationId
people = []  #firstName, lastName, born, died, gender, winnerId
organizations = [] #name, winnerId
countries = [] #name, countryCode
winning = [] #text, motivationId

prizeId = 0

for jsonobj in laureatesList:
    identifier = jsonobj['id']
    winners.append({})
    winners[-1]['id'] = identifier
    if jsonobj['gender'] == "org":
        organizations.append({})
        winners[-1]['type'] = 'organisation'
        if 'born' in jsonobj:
            organizations[-1]['founded'] = jsonobj['born'].split('-', 1)[0]
        organizations[-1]['name'] = jsonobj['firstname']
        organizations[-1]['id'] = jsonobj['id']
    else:
        winners[-1]['type'] = 'person'
        people.append({})
        if 'firstname' in jsonobj:
            people[-1]['firstname'] = jsonobj['firstname']
        if 'surname' in jsonobj:
            people[-1]['lastname'] = jsonobj['surname']
        people[-1]['born'] = jsonobj['born'].split('-', 1)[0]
        people[-1]['died'] = jsonobj['died'].split('-', 1)[0]
        people[-1]['gender'] = jsonobj['gender']
        people[-1]['id'] = jsonobj['id']
        if 'diedCountry' in jsonobj:
            people[-1]['deathCountry'] = jsonobj['diedCountry']
        if 'bornCountry' in jsonobj:
            people[-1]['bornCountry'] = jsonobj['bornCountry']
            
#     for key in jsonobj:
#         if key == 'prizes':
#             for prizeobj in jsonobj['prizes']:
#                 winning.append({}) #OBS behövs inte, ersätts av motivations
#                 winning[-1]['winnerId'] = jsonobj['id']
#                 winning[-1]['motivation'] = prizeobj['motivation']

prizeId = 0
for jsonobj in prizeList:
    prizeId += 1
    prizes.append({})
    prizes[-1]['category'] = jsonobj['category']
    prizes[-1]['year'] = jsonobj['year']
    prizes[-1]['id'] = prizeId
    if 'overallMotivation' in jsonobj:
        prizes[-1]['overallMotivation'] = jsonobj['overallMotivation']
    if 'laureates' in jsonobj: #OBS vissa år delades inget nobelpris ut!!    
        winners_basic_info = jsonobj['laureates']
        for winner in winners_basic_info:
            winning.append({})
            if 'motivation' in winner:
                winning[-1]['motivation'] = winner['motivation']
            winning[-1]['winnerId'] = winner['id']
            winning[-1]['prizeId'] = prizeId
            
import pandas as pd
pd.set_option('display.max_rows', None)

winners_frame = pd.DataFrame.from_dict(winners)
prizes_frame = pd.DataFrame.from_dict(prizes)
organizations_frame = pd.DataFrame.from_dict(organizations)
countries_frame = pd.DataFrame.from_dict(countries)
winning_frame = pd.DataFrame.from_dict(winning)
people_frame = pd.DataFrame.from_dict(people)

In [10]:
winning_frame

,motivation,winnerId,prizeId
0,"""for the development of click chemistry and bi...",1015,1
1,"""for the development of click chemistry and bi...",1016,1
2,"""for the development of click chemistry and bi...",743,1
3,"""for research on banks and financial crises""",1021,2
4,"""for research on banks and financial crises""",1022,2
5,"""for research on banks and financial crises""",1023,2
6,"""for the courage and clinical acuity with whic...",1017,3
7,"""The Peace Prize laureates represent civil soc...",1018,4
8,"""The Peace Prize laureates represent civil soc...",1019,4
9,"""The Peace Prize laureates represent civil soc...",1020,4


# Create tables in the database

In [38]:
try:
    tableName = "prizes"
    create_table_query = '''CREATE TABLE '''+ tableName +''' 
              (prizeid           NUMERIC,
               year          NUMERIC,
               category          TEXT,
               overallmotivation          TEXT,
               PRIMARY KEY (id)
               ); '''

    #Execute this command (SQL Query)
    cursor.execute(create_table_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("Table ("+ tableName +") created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print(error)

Table (prizes) created successfully in PostgreSQL 


In [39]:
try:
    tableName = "organizations"
    create_table_query = '''CREATE TABLE '''+ tableName +''' 
              (winnerid           NUMERIC,
               founded          NUMERIC,
               name          TEXT,
               PRIMARY KEY (id)
               ); '''

    #Execute this command (SQL Query)
    cursor.execute(create_table_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("Table ("+ tableName +") created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print(error)

Table (organizations) created successfully in PostgreSQL 


In [46]:
try:
    tableName = "people"
    create_table_query = '''CREATE TABLE '''+ tableName +''' 
              (winnerid           NUMERIC,
               firstname          TEXT,
               lastname          TEXT,
               born          NUMERIC,
               died          NUMERIC,
               gender          TEXT,
               borncountry          TEXT,
               deathcountry          TEXT,
               PRIMARY KEY (id)
               ); '''

    #Execute this command (SQL Query)
    cursor.execute(create_table_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("Table ("+ tableName +") created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print(error)

Table (people) created successfully in PostgreSQL 


In [41]:
try:
    tableName = "winners"
    create_table_query = '''CREATE TABLE '''+ tableName +''' 
              (winnerid           NUMERIC,
               type          TEXT,
               PRIMARY KEY (id)
               ); '''

    #Execute this command (SQL Query)
    cursor.execute(create_table_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("Table ("+ tableName +") created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print(error)

Table (winners) created successfully in PostgreSQL 


In [42]:
try:
    tableName = "winning"
    create_table_query = '''CREATE TABLE '''+ tableName +''' 
              (winnerid           NUMERIC,
               prizeid          NUMERIC,
               motivation          TEXT,
               PRIMARY KEY (winnerid, prizeid)
               ); '''

    #Execute this command (SQL Query)
    cursor.execute(create_table_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("Table ("+ tableName +") created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print(error)

Table (winning) created successfully in PostgreSQL 


# Convert pandas frames to SQL insert statements

In [43]:
#sql = "INSERT INTO `winning` (`motivation`, `winnerId`, `prizeId`) VALUES (%s, %s, %s)"
# Execute the query
#cursor.execute(sql, ("in recognition of the extraordinary serv", "1", "2"))

#winning_frame.to_sql('winning', con, if_exists='replace', index = False)

import re

def get_insert_query_from_df(df, dest_table):

    insert = """
    INSERT INTO "{dest_table}" (
        """.format(dest_table=dest_table)

    columns_string = str(list(df.columns))[1:-1]
    columns_string = re.sub(r' ', '\n        ', columns_string)
    columns_string = re.sub(r'\'', '', columns_string)

    values_string = ''

    for row in df.itertuples(index=False,name=None):
        values_string += re.sub(r'nan', 'null', str(row))
        values_string += ',\n'

    return insert + columns_string + ')\n     VALUES\n' + values_string[:-2] + ';'

In [44]:
text = get_insert_query_from_df(winners_frame, "winners")
print(text)


    INSERT INTO "winners" (
        id,
        type)
     VALUES
('1', 'person'),
('2', 'person'),
('3', 'person'),
('4', 'person'),
('5', 'person'),
('6', 'person'),
('8', 'person'),
('9', 'person'),
('10', 'person'),
('11', 'person'),
('12', 'person'),
('13', 'person'),
('14', 'person'),
('15', 'person'),
('16', 'person'),
('17', 'person'),
('18', 'person'),
('19', 'person'),
('20', 'person'),
('21', 'person'),
('22', 'person'),
('23', 'person'),
('24', 'person'),
('25', 'person'),
('26', 'person'),
('27', 'person'),
('28', 'person'),
('29', 'person'),
('30', 'person'),
('31', 'person'),
('32', 'person'),
('33', 'person'),
('34', 'person'),
('35', 'person'),
('36', 'person'),
('37', 'person'),
('38', 'person'),
('39', 'person'),
('40', 'person'),
('41', 'person'),
('42', 'person'),
('43', 'person'),
('44', 'person'),
('45', 'person'),
('46', 'person'),
('47', 'person'),
('48', 'person'),
('49', 'person'),
('50', 'person'),
('51', 'person'),
('52', 'person'),
('53', 'person'),
('54'

In [45]:
text = get_insert_query_from_df(winning_frame, "winning")
print(text)


    INSERT INTO "winning" (
        motivation,
        winnerId,
        prizeId)
     VALUES
('"for the development of click chemistry and bioorthogonal chemistry"', '1015', 1),
('"for the development of click chemistry and bioorthogonal chemistry"', '1016', 1),
('"for the development of click chemistry and bioorthogonal chemistry"', '743', 1),
('"for research on banks and finullcial crises"', '1021', 2),
('"for research on banks and finullcial crises"', '1022', 2),
('"for research on banks and finullcial crises"', '1023', 2),
('"for the courage and clinical acuity with which she uncovers the roots, estrangements and  collective restraints of personal memory"', '1017', 3),
('"The Peace Prize laureates represent civil society in their home countries. They have for many years promoted the right to criticise power and protect the fundamental rights of citizens. They have made an outstanding effort to document war crimes, human right abuses and the abuse of power. Together they demonstr

In [35]:
text = get_insert_query_from_df(prizes_frame, "prizes")
print(text)


    INSERT INTO "prizes" (
        category,
        year,
        id,
        overallMotivation)
     VALUES
('chemistry', '2022', 1, null),
('economics', '2022', 2, null),
('literature', '2022', 3, null),
('peace', '2022', 4, null),
('physics', '2022', 5, null),
('medicine', '2022', 6, null),
('chemistry', '2021', 7, null),
('economics', '2021', 8, null),
('literature', '2021', 9, null),
('peace', '2021', 10, null),
('physics', '2021', 11, '"for groundbreaking contributions to our understanding of complex physical systems"'),
('medicine', '2021', 12, null),
('chemistry', '2020', 13, null),
('economics', '2020', 14, null),
('literature', '2020', 15, null),
('peace', '2020', 16, null),
('physics', '2020', 17, null),
('medicine', '2020', 18, null),
('chemistry', '2019', 19, null),
('economics', '2019', 20, null),
('literature', '2019', 21, null),
('peace', '2019', 22, null),
('physics', '2019', 23, '"for contributions to our understanding of the evolution of the universe and Earthâ€™s 

In [36]:
text = get_insert_query_from_df(people_frame, "people")
print(text)


    INSERT INTO "people" (
        firstname,
        lastname,
        born,
        died,
        gender,
        id,
        deathCountry,
        bornCountry)
     VALUES
('Wilhelm Conrad', 'Röntgen', '1845', '1923', 'male', '1', 'Germany', 'Prussia (now Germany)'),
('Hendrik A.', 'Lorentz', '1853', '1928', 'male', '2', 'the Netherlands', 'the Netherlands'),
('Pieter', 'Zeeman', '1865', '1943', 'male', '3', 'the Netherlands', 'the Netherlands'),
('Henri', 'Becquerel', '1852', '1908', 'male', '4', 'France', 'France'),
('Pierre', 'Curie', '1859', '1906', 'male', '5', 'France', 'France'),
('Marie', 'Curie', '1867', '1934', 'female', '6', 'France', 'Russian Empire (now Poland)'),
('Lord', 'Rayleigh', '1842', '1919', 'male', '8', 'United Kingdom', 'United Kingdom'),
('Philipp', 'Lenard', '1862', '1947', 'male', '9', 'Germany', 'Hungary (now Slovakia)'),
('J.J.', 'Thomson', '1856', '1940', 'male', '10', 'United Kingdom', 'United Kingdom'),
('Albert A.', 'Michelson', '1852', '1931', 'mal

In [37]:
text = get_insert_query_from_df(organizations_frame, "organizations")
print(text)


    INSERT INTO "organizations" (
        founded,
        name,
        id)
     VALUES
('1873', 'Institute of International Law', '467'),
('1891', 'Permanent International Peace Bureau', '477'),
('1863', 'International Committee of the Red Cross', '482'),
('1921', 'Nansen International Office for Refugees', '503'),
('1647', 'Friends Service Council', '508'),
('1917', 'American Friends Service Committee', '509'),
('1950', 'Office of the United Nations High Commissioner for Refugees', '515'),
('1919', 'League of Red Cross Societies', '523'),
('1946', "United Nations Children's Fund", '525'),
('1919', 'International Labour Organization', '527'),
('1961', 'Amnesty International', '537'),
('1980', 'International Physicians for the Prevention of Nuclear War', '547'),
('1948', 'United Nations Peacekeeping Forces', '550'),
('1957', 'Pugwash Conferences on Science and World Affairs', '561'),
('1992', 'International Campaign to Ban Landmines', '564'),
('1971', 'Doctors Without Borders', '568'